In [62]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.optimizers import Adam

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.utils import class_weight


In [63]:
BASE_DIR = '../../'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 2000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
Found 400000 word vectors.


In [64]:
t = pd.read_csv("./reports/data.csv")
t = t.dropna()

In [90]:
def check(a):
    if a in ['dp','geometry', 'greedy', 'combinatorics', 'datastructures'
            , 'games', 'graphs', 'numbertheory', 'probabilities', 'strings'] :
        return True
    else:
        return False


In [91]:
temp = t['labels'].tolist()
res = [text_to_word_sequence(a) for a in temp]
res = [[a for a in line if check(a) ] for line in res]

In [92]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(res)

feature = "problem-statement"

l =t['ps']= t['problem-statement'].tolist()
l2 = t['os']= t['output-spec'].tolist()
res1 = []
res2 = []
regex = re.compile("^a-zA-Z0-9\ '" )
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        res1.append(clean_s(l[i]))
    except:
        res1.append("")
for i in range(len(l2)):
    try:
        res2.append(clean_s(l2[i]))
    except:
        res2.append("")
texts = res1

In [93]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(output['ps'].astype(str) + output['os'])
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 8547 unique tokens.


In [94]:
print(labels.shape)

(4277, 10)


In [95]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

Preparing embedding matrix.


In [96]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [97]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


In [100]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='sigmoid')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
validation_data=(x_val, y_val))
y_pred = model.predict(x_train)

Train on 3422 samples, validate on 855 samples
Epoch 1/10
3422/3422 [==============================] - 3s 990us/step - loss: 0.3233 - acc: 0.8888 - val_loss: 0.2799 - val_acc: 0.9142
Epoch 2/10
3422/3422 [==============================] - 3s 778us/step - loss: 0.2823 - acc: 0.9153 - val_loss: 0.2749 - val_acc: 0.9142
Epoch 3/10
3422/3422 [==============================] - 3s 781us/step - loss: 0.2739 - acc: 0.9153 - val_loss: 0.2697 - val_acc: 0.9142
Epoch 4/10
3422/3422 [==============================] - 3s 783us/step - loss: 0.2709 - acc: 0.9153 - val_loss: 0.2635 - val_acc: 0.9142
Epoch 5/10
3422/3422 [==============================] - 3s 783us/step - loss: 0.2640 - acc: 0.9153 - val_loss: 0.2678 - val_acc: 0.9142
Epoch 6/10
3422/3422 [==============================] - 3s 783us/step - loss: 0.2591 - acc: 0.9152 - val_loss: 0.2640 - val_acc: 0.9143
Epoch 7/10
3422/3422 [==============================] - 3s 785us/step - loss: 0.2507 - acc: 0.9156 - val_loss: 0.2573 - val_acc: 0.9142
E

In [109]:
y_pred = model.predict(x_val)

In [115]:
[True for a in line for line in y_pred if a >= 0.3]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,